In [1]:
!pip install -r /content/drive/MyDrive/textG/requirements.txt

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
import os
import sys
sys.path.append('/content/drive/MyDrive/textG')
print(os.getcwd())



/content


In [3]:
import random
import torch
from torch.utils.data import DataLoader # 데이터로더
from gluonnlp.data import SentencepieceTokenizer 
from model.torch_gpt2 import GPT2Config, GPT2LMHeadModel # model폴더의 torch_gpt2.py의 
# from transformers.configuration_gpt2 import GPT2Config 를 from transformers import GPT2Config로 변경해 오류 잡음
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from util.data import FairyDataset
import gluonnlp
from tqdm import tqdm


In [4]:
ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='/content/drive/MyDrive/textG/kogpt2/' # KoGPT-2 모델 다운로드 경로
save_path = '/content/drive/MyDrive/textG/checkpoint/'
use_cuda = True # Colab내 GPU 사용을 위한 값

kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 51200
}

In [5]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
# Device 설정
device = torch.device(ctx)
# 저장한 Checkpoint 불러오기
checkpoint = torch.load(save_path+'gt_checkpoint_2.tar', map_location=device)
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.eval()
vocab_b_obj = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='<s>', eos_token='</s>', unk_token='<unk>',  pad_token='<pad>', mask_token='<mask>') 


Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [6]:

from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
# Device 설정
device = torch.device(ctx)
# 저장한 Checkpoint 불러오기
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/narrativeKoGPT2/checkpoint/aesop_checkpoint_2_110_3.tar', map_location=device)
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.eval()
vocab_b_obj = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='<s>', eos_token='</s>', unk_token='<unk>',  pad_token='<pad>', mask_token='<mask>') 


KeyboardInterrupt: ignored

In [6]:

model, vocab = kogpt2model, vocab_b_obj.get_vocab()
tok = vocab_b_obj.tokenize
revers_vocab = {v:k for k,v in vocab.items()}


In [7]:
import torch
from transformers import GPT2LMHeadModel


text = '나무꾼 : "나무꾼이 나무를 하고 있었어요"하던 여우는 그나무를 만들어 시장에 내다팔고 있었어요.'
input_ids = vocab_b_obj.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                         max_length=128,
                         repetition_penalty=2.0,
                         pad_token_id=vocab_b_obj.pad_token_id,
                         eos_token_id=vocab_b_obj.eos_token_id,
                         bos_token_id=vocab_b_obj.bos_token_id,
                         use_cache=True)
generated = vocab_b_obj.decode(gen_ids[0,:].tolist())
print(generated)

나무꾼 : "나무꾼이 나무를 하고 있었어요"하던 여우는 그나무를 만들어 시장에 내다팔고 있었어요. 같았고, 특정 아내가 있었다. 지금은 그것이 모든 사람들들은 그녀와 함께 있었고, 그 어린 딸에게 불길에 예뻤던 부드럽게 물었다. 그것은 길다는 생각이 들어가 잠든 어느 날 밤새 일어나 집을 떠나 성 동화를 찾으러 가는 게 없다고 생각했기 때문에 그는 두 사람이었고, 그녀는 말했다. 그래서 그에게 일어난 것은 나비를 가지고 있는 것처럼 항상 손을 괜찮지만 물이 거의 없었고, 산책을 달라고 부탁했고, 이를 지시했다 그러나 그들은 자신들이 선박이 코나는 자신의 작품에 나갔다가 아주 좋은 일은 어떻게 되라"고


In [17]:
!nvidia-smi

Tue Oct  5 03:57:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    33W / 250W |   5823MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
torch.cuda.is_available()

True

In [19]:
!python /content/drive/MyDrive/textG/main.py

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
나뭇꾼 : 럼킨 오래전 가난한 한때 옷을 살았고, 그 남자는 남자와 아내가 있었다. 몇 년 남자가 살았다. 그들은 세계 둘 다 더 이상 행운을 시도해 하지 않았다. 그러나 그녀의 어머니는 그를 만나차를하고 아름다운 여성들었고, 그들이 자신들의 사업을 돌아다녔다 "그렇게 해야 할지 몰랐던" 딸이 앉아 쉬고 놀랐고, 좋은 사람이었고, 그녀는 할머니는 너를 주었고, 어떻게 네가 될까"라고 말하며 모든 일이 일어날지 않았다"며 "그리고다"고 말한 대로 했고, 노인을 잘라내기 위해 잠을 잘생겼을 때, 그의 방에서 물을 가져와 주었다. "이제 공주와 사랑에 빠뜨렸다는 것이다"라고 그는 집으로 돌아갔는데, 내


In [9]:
!cd /content/drive/MyDrive/textG/

In [11]:
/content/drive/MyDrive/textG/checkpoint

drive  sample_data
